# Newton Methods in Scipy

## Learning Objectives

After studying this notebook, completing the activities, and asking questions in class, you should be able to:
* Use numpy to solve the flash example problem from the [previous notebook](../04-publish/05-System-of-Equations-Newton-Method.ipynb).

In [1]:
# load libraries
import numpy as np
from scipy import optimize

## Using ``scipy`` instead

`numpy` and `scipy` offer a few different implementations of Newton's method. However, we found these to be unreliable in the past. Instead, we recommend either using the Newton solver we put together in the [last notebook](../04-publish/05-System-of-Equations-Newton-Method.ipynb) or Pyomo (future notebook).

However, we will show how this is done using `scipy.optimize.newton` in this notebook.

In [2]:
# documentation for Newton's method in scipy
help(optimize.newton)

Help on function newton in module scipy.optimize._zeros_py:

newton(func, x0, fprime=None, args=(), tol=1.48e-08, maxiter=50, fprime2=None, x1=None, rtol=0.0, full_output=False, disp=True)
    Find a zero of a real or complex function using the Newton-Raphson
    (or secant or Halley's) method.
    
    Find a zero of the scalar-valued function `func` given a nearby scalar
    starting point `x0`.
    The Newton-Raphson method is used if the derivative `fprime` of `func`
    is provided, otherwise the secant method is used. If the second order
    derivative `fprime2` of `func` is also provided, then Halley's method is
    used.
    
    If `x0` is a sequence with more than one item, `newton` returns an array:
    the zeros of the function from each (scalar) starting point in `x0`.
    In this case, `func` must be vectorized to return a sequence or array of
    the same shape as its first argument. If `fprime` (`fprime2`) is given,
    then its return must also have the same shape: eac

## Polynomial Example Revisited Using Scipy

$$ c(x) = 3 x^3 + 2 x^2 - 5 x - 20 $$

![Newton Polynomial](../../media/newton_polynomial.png)
    
In the following cells we will demonstrate the use of Scipy to perform the Newton-Raphson method for the polynomial example from [this previous notebook](../04-publish/02-Newton-Raphson-Method-in-One-Dimension.ipynb).

First, we need to run the code we already developed for the polynomial function and its derivative.

In [3]:
def nonlinear_function(x):
    ''' compute a nonlinear function for demonstration
    Arguments:
        x: scalar
    Returns:
        c(x): scalar
    '''
    return 3*x**3 + 2*x**2 - 5*x-20

def Dnonlinear_function(x):
    ''' compute 1st derivative of nonlinear function for demonstration
    Arguments:
        x: scalar
    Returns:
        c'(x): scalar
    '''
    return 9*x**2 + 4*x - 5

Next, we need to provide an initial guess as we did before we the code we deveoped for Newton's Method.

In [4]:
guess = 1

Now we can run Newton's Method through `scipy.optimize.newton`.

In [5]:
polysln = optimize.newton(func=nonlinear_function, x0 = guess, fprime=Dnonlinear_function)
print("The root is found at",polysln)

The root is found at 1.9473052357731322


## Flash Problem Revisted Using Scipy

Recall the flash problem from [this previous notebook](../04-publish/01-Modeling-Systems-of-Nonlinear-Equations.ipynb) that we solved in the [last notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb) using Newton's Method.  In the following cells we will demonstrate the use of Scipy to perform the Newton-Raphson method.

![flash](../../media/flash-system.png)

**Parameters (given)**:
* $F$ feed inlet flowrate, mol/time or kg/time
* $z_1$ composition of species 1 in feed, mol% or mass%
* $z_2$ composition of species 2 in feed, mol% or mass%
* $K_1$ partion coefficient for species 1, mol%/mol% or mass% / mass%
* $K_2$ partion coefficient for species 2, mol%/mol% or mass% / mass%

**Variables (unknown)**:
* $L$ liquid outlet flowrate, mol/time or kg/time
* $x_1$ composition of species 1 in liquid, mol% or mass%
* $x_2$ composition of species 2 in liquid, mol% or mass%
* $V$ vapor outlet flowrate, mol/time or kg/time
* $y_1$ composition of species 1 in vapor, mol% or mass%
* $y_2$ composition of species 2 in vapor, mol% or mass%

How to solve the flash problem and other multidimensional problem with $n$ unknown variables and $n$ nonlinear equations?

### System of Equations in Canonical Form

$$\mathbf{F}(\mathbf{x}) = \begin{pmatrix}
L + V - F\\
Vy_1 + L x_1 - F z_1 \\
V y_2 + L x_2 - F z_2 \\
y_1 - K_1 x_1 \\
y_2 - K_2 x_2 \\
y_1 + y_2 - x_1 - x_2
\end{pmatrix},$$

with $\mathbf{x} = (L, V, x_1, x_2, y_1, y_2).$

First run the code below to define the system of nonlinear equations given in the [previous notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb).

In [6]:
# nonlinear canonical system of equations
def my_f(x,verbose=True):
    ''' Nonlinear system of equations in conancial form F(x) = 0
    Copied from previous notebook.
    
    Arg:
        x: vector of variables
        
    Returns:
        r: residual, F(x)
    
    '''

    # Initialize residuals
    r = np.zeros(6)
    
    # given data
    F = 1.0
    z1 = 0.5
    z2 = 0.5
    K1 = 3
    K2 = 0.05
    
    # copy values from x to more meaningful names
    L = x[0]
    V = x[1]
    x1 = x[2]
    x2 = x[3]
    y1 = x[4]
    y2 = x[5]
    
    # equation 1: overall mass balance
    r[0] = V + L - F
    
    # equations 2 and 3: component mass balances
    r[1] = V*y1 + L*x1 - F*z1
    r[2] = V*y2 + L*x2 - F*z2
    
    # equation 4 and 5: equilibrium
    r[3] = y1 - K1*x1
    r[4] = y2 - K2*x2
    
    # equation 6: summation
    r[5] = (y1 + y2) - (x1 + x2)
    # This is known as the Rachford-Rice formulation for the summation constraint
    
    if verbose:
        print("Evaluating my_f at x=",x)
        print("Residuals r=",r,"\n")
    
    return r

Next, we need to define the initial guess as we did with our Newton System in the [previous notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb).

In [7]:
# initial guess
x0 = np.array([0.5, 0.5, 0.55, 0.45, 0.65, 0.35])

Now we can run Newton's Method through `scipy.optimize.newton`.  Note that the secant method is used if a derivative is not given to the `scipy.optimize.newton` function, as is the case below.

In [8]:
xsln1 = optimize.newton(func=my_f, x0=x0)
print("Solution using scipy:\n",xsln1)

Evaluating my_f at x= [0.5  0.5  0.55 0.45 0.65 0.35]
Residuals r= [ 0.      0.1    -0.1    -1.      0.3275  0.    ] 

Evaluating my_f at x= [0.50001024 0.50001024 0.55001058 0.4500099  0.65001127 0.35000922]
Residuals r= [ 2.04854981e-05  1.00023217e-01 -9.99822457e-02 -1.00002049e+00
  3.27508723e-01  0.00000000e+00] 

Evaluating my_f at x= [ 0.5         0.45588278  0.60961473 -0.03333333  0.22700587  0.35000461]
Residuals r= [-4.41172218e-02 -9.17045694e-02 -3.57105593e-01 -1.60183831e+00
  3.51671276e-01  7.29086387e-04] 

Evaluating my_f at x= [0.50001024 0.47698921 0.52683355 1.25316514 6.38359249 0.35000922]
Residuals r= [-0.02300055  2.80832693  0.29354602  4.80309183  0.28735096  4.95360302] 

Evaluating my_f at x= [ 0.50002139  0.4565502   0.56418086  0.28841296 33.88813918  0.35000461]
Residuals r= [-0.04342841 15.25373936 -0.19599268 32.19559661  0.33558396 33.38554997] 

Evaluating my_f at x= [   0.49999768    0.48160131    0.54922842    1.42232795 -157.47638155
    0.3500

C:\Users\ebrea\anaconda3\envs\jbook\lib\site-packages\scipy\optimize\_zeros_py.py:466: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)


The correct answer should be <code> [0.72368421 0.27631579 0.3220339  0.6779661  0.96610169 0.03389831] </code>

However, we see that all of the values are incorrect and the third and fifth values diverged.  This illustrates why we prefer to use our own code for Newton's method over scipy.

Now let's try giving it a second guess of x1 that is close to what we know to be the correct answer.

In [9]:
x1 = np.array([0.72, 0.27, 0.32, 0.67, 0.96, 0.03])
xsln2 = optimize.newton(func=my_f, x0=x0, x1=x1)
print("Solution using scipy:\n",xsln2)

Evaluating my_f at x= [0.5  0.5  0.55 0.45 0.65 0.35]
Residuals r= [ 0.      0.1    -0.1    -1.      0.3275  0.    ] 

Evaluating my_f at x= [0.50001024 0.50001024 0.55001058 0.4500099  0.65001127 0.35000922]
Residuals r= [ 2.04854981e-05  1.00023217e-01 -9.99822457e-02 -1.00002049e+00
  3.27508723e-01  0.00000000e+00] 

Evaluating my_f at x= [ 0.5         0.45588278  0.60961473 -0.03333333  0.22700587  0.35000461]
Residuals r= [-4.41172218e-02 -9.17045694e-02 -3.57105593e-01 -1.60183831e+00
  3.51671276e-01  7.29086387e-04] 

Evaluating my_f at x= [0.50001024 0.47698921 0.52683355 1.25316514 6.38359249 0.35000922]
Residuals r= [-0.02300055  2.80832693  0.29354602  4.80309183  0.28735096  4.95360302] 

Evaluating my_f at x= [ 0.50002139  0.4565502   0.56418086  0.28841296 33.88813918  0.35000461]
Residuals r= [-0.04342841 15.25373936 -0.19599268 32.19559661  0.33558396 33.38554997] 

Evaluating my_f at x= [   0.49999768    0.48160131    0.54922842    1.42232795 -157.47638155
    0.3500

Even adding a second guess closer to the correct root doesn't seem to help.

Let's try one more thing.  Now, we're going to call a finite difference function for the derivative so that we aren't defaulting to the secant method with scipy.

In [10]:
dx = 0.001
central = lambda x:(my_f(x+dx) - my_f(x-dx))/2/dx
xsln3 = optimize.newton(func=my_f, fprime=central, x0=x1)
print("Solution using scipy:\n",xsln3)

Evaluating my_f at x= [0.72 0.27 0.32 0.67 0.96 0.03]
Residuals r= [-0.01   -0.0104 -0.0095  0.     -0.0035  0.    ] 

Evaluating my_f at x= [0.721 0.271 0.321 0.671 0.961 0.031]
Residuals r= [-0.008    -0.008128 -0.007808 -0.002    -0.00255   0.      ] 

Evaluating my_f at x= [0.719 0.269 0.319 0.669 0.959 0.029]
Residuals r= [-0.012    -0.012668 -0.011188  0.002    -0.00445   0.      ] 

Evaluating my_f at x= [0.725      0.2745815  0.3256213  0.67       0.96368421 0.03      ]
Residuals r= [-0.0004185   0.0006853  -0.00601256 -0.01317969 -0.0035     -0.00193709] 

Evaluating my_f at x= [0.726      0.2755815  0.3266213  0.671      0.96468421 0.031     ]
Residuals r= [ 0.0015815   0.00297618 -0.00431097 -0.01517969 -0.00255    -0.00193709] 

Evaluating my_f at x= [0.724      0.2735815  0.3246213  0.669      0.96268421 0.029     ]
Residuals r= [-0.0024185  -0.00160159 -0.00771014 -0.01117969 -0.00445    -0.00193709] 

Evaluating my_f at x= [0.72520925 0.2742821  0.32915897 0.66341015 0.9

C:\Users\ebrea\anaconda3\envs\jbook\lib\site-packages\scipy\optimize\_zeros_py.py:458: RuntimeWarning: some derivatives were zero
  warnings.warn(msg, RuntimeWarning)


Again, we see that the solution is incorrect. From this exploration with `scipy.optimize.newton` we can see that it is often better to use the Newton code we developed in the [previous notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb).